In [1]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('breast-cancer-wisconsin.data', header=None)
# data pre-processing
# take a look of the dataset, we can find the scalar of the first feature is so huge,so its better to do data pre-processing
print(dataset.dtypes)
# we find there are two col values are objects. we need to transform them into numbers
# after checking the dataset, we can find there are some missing data in col7

def find_missingdatarow_number(dataset):
    n = 0
    for i in range(data_num):
        if dataset.loc[i][6] == '?':
            n +=1 
    return n
data_num = dataset.shape[0]
missing_num = find_missingdatarow_number(dataset)
print("missing_num =",missing_num)
print("missing_rate =",missing_num/data_num)

0      int64
1      int64
2      int64
3      int64
4      int64
5      int64
6     object
7      int64
8      int64
9      int64
10     int64
dtype: object
missing_num = 16
missing_rate = 0.022889842632331903


In [2]:
# the missing rows rate is very low. so we can just delete the rows with missing values
def delete_missingdatarow_number(dataset):
    for i in range(data_num):
        if dataset.loc[i][6] == '?':
            dataset = dataset.drop(index = [i])
    return dataset
dt = delete_missingdatarow_number(dataset)
dt = dt.reset_index(drop = True)
print(dt)

          0   1   2   3   4   5   6   7   8   9   10
0    1000025   5   1   1   1   2   1   3   1   1   2
1    1002945   5   4   4   5   7  10   3   2   1   2
2    1015425   3   1   1   1   2   2   3   1   1   2
3    1016277   6   8   8   1   3   4   3   7   1   2
4    1017023   4   1   1   3   2   1   3   1   1   2
..       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
678   776715   3   1   1   1   3   2   1   1   1   2
679   841769   2   1   1   1   2   1   1   1   1   2
680   888820   5  10  10   3   7   3   8  10   2   4
681   897471   4   8   6   4   3   4  10   6   1   4
682   897471   4   8   8   5   4   5  10   4   1   4

[683 rows x 11 columns]


In [49]:
# transform obj data to numbers
dt[6] = dt[6].astype(int)

# standardlize 
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
dt = scaler.fit_transform(dt)

dt = pd.DataFrame(dt)
# split data into train_test 
from sklearn.model_selection import train_test_split
train_dt,test_dt= train_test_split(dt, test_size=0.2)
train_dt = train_dt.reset_index(drop = True)
test_dt = test_dt.reset_index(drop = True)



In [85]:
from sklearn.model_selection import KFold

n_t = train_dt.shape[0]
n_features = train_dt.shape[1]
kf = KFold(n_splits=5,shuffle=True)
# k-fold validation

def sigmoid(a):
    return 1 / (1 + np.exp(float(-a)))

# 
five_folds_mean_df_benign = np.zeros([n_features-1]) 
five_folds_mean_df_malignant = np.zeros([n_features-1])  
            
        
        

In [86]:

for train_index, val_index in kf.split(train_dt):
        tra_x,val_x = train_dt.loc[train_index],train_dt.loc[val_index]
        
        train_x = tra_x[[0,1,2,3,4,5,6,7,8,9]]
        
        x_train_benign = tra_x.loc[tra_x.loc[:, 10] < 0 ][[0,1,2,3,4,5,6,7,8,9]]  #benign dataset
        x_train_benign = x_train_benign.reset_index(drop = True)
        x_train_malignant = tra_x.loc[tra_x.loc[:, 10] > 0][[0,1,2,3,4,5,6,7,8,9]]
        x_train_malignant = x_train_malignant.reset_index(drop = True)
        benign_num =  x_train_benign.shape[0]
        malignant_num =  x_train_malignant.shape[0]
        
        
        cov_ = np.cov(train_x.T)  #cov matrix
        
        mean_df_benign = np.mean(x_train_benign)   #mean vector
        mean_df_malignant = np.mean(x_train_malignant) #mean vector

#       accumulative total of means and cov for calculate avg ones later 
        five_folds_mean_df_benign += mean_df_benign
        five_folds_mean_df_malignant += mean_df_malignant
        five_folds_cov_ += cov_
#       P(C)
        p_c1 = benign_num/(benign_num+malignant_num)
        p_c2 = malignant_num/(benign_num+malignant_num)
        
        val_x = val_x.reset_index(drop = True)
        val_num = val_x.shape[0]
        right_num = 0
        for i in range(val_num):
            x_u1 = val_x.loc[i][[0,1,2,3,4,5,6,7,8,9]] - mean_df_benign
            x_u2 = val_x.loc[i][[0,1,2,3,4,5,6,7,8,9]] - mean_df_malignant
            t1 = (x_u1.T).dot(cov_).dot(x_u1)
            t2 = (x_u2.T).dot(cov_).dot(x_u2)
            a = (0.5 * t2 - 0.5* t1) + np.log(p_c1/p_c2)
            p_c1_x = sigmoid(a)
            if( p_c1_x < 0.5 and val_x.loc[i][10] > 0):
                right_num+= 1
            if( p_c1_x > 0.5 and val_x.loc[i][10] < 0):
                right_num+= 1
        
        print("accuracy on validation dataset is :",right_num/val_num)
                

accuracy on validation dataset is : 0.9727272727272728
accuracy on validation dataset is : 0.9541284403669725
accuracy on validation dataset is : 0.944954128440367
accuracy on validation dataset is : 0.9541284403669725
accuracy on validation dataset is : 0.9724770642201835


In [87]:
# get two mean vector and a cov matrix from 5-fold cross validation training
avg_mean_df_benign = five_folds_mean_df_benign/5
avg_mean_df_malignant = five_folds_mean_df_malignant/5
avg_cov_ = five_folds_cov_/5


In [88]:
# This is the test dataset, the col 10 denotes to whether have the disease 
print(test_dt)

           0         1         2         3         4         5         6   \
0   -0.290744 -0.866417 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   
1    0.077284 -1.221191 -0.049236 -0.072069 -0.290016 -0.555608 -0.557620   
2    0.328186 -0.156869 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   
3    0.429462  1.262227  0.277252  0.262783 -0.639366  1.245011  1.793335   
4    0.140837 -1.221191 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   
..        ...       ...       ...       ...       ...       ...       ...   
132  0.170406 -0.511643 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   
133  0.431689 -1.221191 -0.702212 -0.741774 -0.639366 -1.005763 -0.557620   
134 -0.516913 -1.221191 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   
135 -0.371334  1.971775  0.930228 -0.072069  1.107381  0.344701  1.793335   
136  0.221003 -0.511643 -0.702212 -0.741774 -0.639366 -0.555608 -0.557620   

           7         8         9         10  
0   -0.181827 -0.612927 -0.34

In [89]:
# recall, precision, and accuracy on malignant class prediction on the test data
test_data_num = test_dt.shape[0]

right_benign_num = 0
wrong_benign_num = 0
right_malignant_num = 0
wrong_malignant_num = 0


for i in range(test_data_num):
    x_u1 = test_dt.loc[i][[0,1,2,3,4,5,6,7,8,9]].values - avg_mean_df_benign
    x_u2 = test_dt.loc[i][[0,1,2,3,4,5,6,7,8,9]].values - avg_mean_df_malignant
    t1 = (x_u1.T).dot(avg_cov_).dot(x_u1)
    t2 = (x_u2.T).dot(avg_cov_).dot(x_u2)
    a = (0.5 * t2 - 0.5* t1) + np.log(p_c1/p_c2)
    p_c1_x = float(sigmoid(a))
    if( p_c1_x < 0.5 and test_dt.loc[i][10] > 0):
        right_malignant_num+= 1
    if( p_c1_x > 0.5 and test_dt.loc[i][10] < 0):
        right_benign_num+= 1
    if( p_c1_x < 0.5 and test_dt.loc[i][10] < 0):
        wrong_malignant_num+= 1
    if( p_c1_x > 0.5 and test_dt.loc[i][10] > 0):
        wrong_benign_num+= 1
        


<ipython-input-85-f695107bbd25>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(float(-a)))


In [92]:
malignant_precision = right_malignant_num/(right_malignant_num + wrong_malignant_num)
malignant_recall = right_malignant_num/ (right_malignant_num + wrong_benign_num)
accuracy = (right_benign_num + right_malignant_num) / test_data_num
print("malignant_precision is:",malignant_precision)
print("malignant_recall is   :",malignant_recall)
print("accuracy is           :",accuracy)

malignant_precision is: 0.9761904761904762
malignant_recall is   : 0.9318181818181818
accuracy is           : 0.9708029197080292
